## Data Import and Cleaning

In [229]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import folium
import folium.plugins
from scipy import stats

In [230]:
data = pd.read_csv('/Users/azel/Documents/Data Sets/DC Data/Parking Enforcement/Combined /Parking Violations.csv')

In [231]:
print()
print(data.info())
print()
print(data.isnull().sum())
print()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14424762 entries, 0 to 14424761
Data columns (total 19 columns):
X                        float64
Y                        float64
OBJECTID                 int64
ROWID_                   float64
DAY_OF_WEEK              object
HOLIDAY                  float64
WEEK_OF_YEAR             float64
MONTH_OF_YEAR            float64
ISSUE_TIME               float64
VIOLATION_CODE           object
VIOLATION_DESCRIPTION    object
LOCATION                 object
RP_PLATE_STATE           object
BODY_STYLE               object
ADDRESS_ID               int64
STREETSEGID              float64
XCOORD                   int64
YCOORD                   int64
TICKET_ISSUE_DATE        object
dtypes: float64(8), int64(4), object(7)
memory usage: 2.0+ GB
None

X                              0
Y                              0
OBJECTID                       0
ROWID_                   2595214
DAY_OF_WEEK              4432946
HOLIDAY                  4432946
WEEK_O

,X,Y,OBJECTID,ROWID_,DAY_OF_WEEK,HOLIDAY,WEEK_OF_YEAR,MONTH_OF_YEAR,ISSUE_TIME,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,BODY_STYLE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE
0,-76.990883,38.901329,24651355,15241.0,THURSDAY,0.0,36.0,9.0,2109.0,P012,DISOBEYING OFFICIAL SIGN,1200 BLOCK I ST NE NORTH SIDE,NY,4D,801937,2090.0,400791,137052,2011-09-01T00:00:00.000Z
1,-77.028853,38.917009,24651356,15242.0,THURSDAY,0.0,36.0,9.0,2158.0,P039,PARK AT EXPIRED METER,1300 BLOCK U ST NW NORTH SIDE,MD,4D,809133,7819.0,397498,138793,2011-09-01T00:00:00.000Z
2,-77.013880,38.888564,24651357,15243.0,THURSDAY,0.0,36.0,9.0,927.0,P036,PARK IN OFFICIAL PARKING PERMIT ONLY SPACE,200 MARYLAND AV SW,DC,TK,801240,1186.0,398796,135635,2011-09-01T00:00:00.000Z
3,-77.054309,38.926666,24651358,15244.0,THURSDAY,0.0,36.0,9.0,107.0,P172,FAILURE TO SECURE DC TAGS,2600 BLOCK GARFIELD ST NW NORTH SID,NJ,4D,806063,5205.0,395291,139866,2011-09-01T00:00:00.000Z
4,-77.023957,38.897877,24651359,15245.0,THURSDAY,0.0,36.0,9.0,1011.0,P199,PARK IN A DESIGNATED ENTRANCE,700 9TH ST NW BLOCK EAST SIDE,DC,NaN,813593,11702.0,397922,136669,2011-09-01T00:00:00.000Z


In [232]:
data = (data.drop(columns = ['OBJECTID','ROWID_','DAY_OF_WEEK','HOLIDAY','WEEK_OF_YEAR','MONTH_OF_YEAR','ISSUE_TIME',
                             'XCOORD','YCOORD','ADDRESS_ID'])
        .rename(columns = {'VIOLATION_DESCRIPTION':'Violation Description','VIOLATION_CODE': 'Violation Code',
                           'STREETSEGID': 'Block ID', 'LOCATION':'Address','RP_PLATE_STATE':'License Plate State', 
                           'BODY_STYLE':'Body Style','TICKET_ISSUE_DATE':'Ticket Issue Date', 'Y': 'Latitude',
                           'X': 'Longitude'}))

na_map = {"P076" : 'ROSA'}
mask = data['Violation Description'].isnull()
data.loc[ mask , 'Violation Description' ] = data.loc[mask, 'Violation Code'].map(na_map)

data['License Plate State'] = data['License Plate State'].fillna('NA')
data['Body Style'] = data['Body Style'].fillna('Other')


In [233]:
meter_violations = ('PARK AT EXPIRED METER','FAIL TO DISPLAY A MULTISPACE METER RECEIPT',
                    'PARK OVERTIME AT A METER','EXPIRATION TIME ON METER RECEIPT LAPSED',
                   'METER, FAIL TO DEPOSIT PAYMENT','EXPIRED HANDICAP METER',
                    'FAIL TO DISPLAY METER RECEIPT IN A LOADING ZONE','NOT PARKED IN A METER SPACE',
                    'OVERSIZED COMM VEHICLE PARKED AT A METER','IND. W/ DISABILITIES METER  NO PROOF OF PAYMENT',
                   'FAIL TO DISPLAY MULTI SPACE RECEIPT BALLPARKEVENT','ILLEGAL AT METER',
                   'FAIL TO DISPLAY MULTI SPACE RECEIPT  BALLPARKEVENT','OVERTIME AT A MULTISPACE METER IN A LOADING ZONE',
                   'FAIL TO PROPERLY DISPLAY MULTISPACE METER RECEIPT','FAIL TO DISPLAY MULTI-SPACE METER RECEIPT PROPERLY',
                   'FAIL TO DISPLAY RESIDENTIAL PARKING PERMIT','FAIL TO DEPOSIT COIN IN METER','OVERSIZED VEHICLE AT METER BALL PARK EVENT')
temp_restrictions = ('NO PARKING STREET CLEANING','NO STOPPING OR STANDING IN PM RUSH HOUR ZONE',
                    'NO STOPPING OR STANDING IN AM RUSH HOUR ZONE','EMERGENCY NO PARKING',
                    'NO PARKING SPECIFIC HOURS','RELOCATE TOW FEE','PARKING ON SNOW EMERGENCY ROUTE WHEN PROHIBITED',
                    'NO PARKING 7 AM TO 6:30 PM')
illegal_location = ('DISOBEYING OFFICIAL SIGN','NO PARKING ANYTIME','UNAUTHORIZED VEHICLE IN LOADING ZONE',
                    'NO STANDING ANYTIME','PARKED WITHIN 25 FEET OF A STOP SIGN','PARK IN A BUS ZONE',
                    'PARK WITHIN 5 FEET OF DRIVEWAY OR ALLEY','VEHICLE ON PRIVATE/PUBLIC PROPERTY WITHOUT CONSENT',
                   'PARK WITHIN 10 FEET OF A FIRE HYDRANT','NO STOPPING ANY TIME',
                   'PARK IN RESERVED SPACE FOR MOTORCYCLES ONLY','PARK ON PUBLIC SPACE','PARK WITH LEFT WHEEL TO THE CURB',
                   'PARK LESS THAN 25 FEET FROM AN INTERSECTION','PARK IN SCHOOL ZONE','PARK LESS THAN 40 FEET FROM AN INTERSECTION',
                    'PARK ON TAXICAB STAND','PARK ON SIDEWALK OR AREA RESERVED FOR PEDESTRIANS','NO STOPPING ANYTIME',
                    'NO STOPPING OR STANDING ANYTIME','GOVERNMENT VEHICLE PARKING ONLY','PARKED ON DESIGNATED VENDOR AREA',
                   'PARK IN RESERVED EMBASSY SPACE','PARK ON OR UNDER AN ELEVATED STRUCTURE','PARK IN A FIRE LANE',
                    'PARK IN MEDIAN, ISLAND, OR SAFETY ZONE','NON-TAXI ON HACK STAND','PARKED IN A RESERVED CAR SHARING SPACE',
                    'OBSTRUCTING ENTRANCE OF PKG GARAGE','ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 1',
                   'PARK ON PRIVATE OR PUBLIC PROPERTY','PARK WITHIN 20 FEET OF A BUS STOP OR ZONE','VALET PARKING/STAGING ZONE',
                   'FAIL TO PARK PARALLEL','VALET PARKING/STAGING ZONE','MOTOR RUNNING UNATTENDED','PARK VEHICLE IN FRONT OF A BARRICADE',
                   'COMMERCIAL MOTOR VEHICLE - NOT OUT OF SERVICE','DANGEROUS VEHICLE ON PRIV/PUBLIC PROP VIOL 1','PARK ON MEDIAN, ISLAND, OR SAFETY ZONE',
                   'OBSTRUCT AN INTERSECTION','PARK BETWEEN ISLAND OR SAFETY ZONE','PARK WITHIN TWENTY FIVE FEET FROM A YIELD SIGN')
obstruct_traffic = ('STAND OR PARK IN ALLEY','STAND OR PARK IN ALLEY','PARK IN A DESIGNATED ENTRANCE','PARK ABREAST OF ANOTHER VEHICLE',
                  'OBSTRUCTING CROSSWALK','STOPPING, STANDING OR PARKING VEHICLE IN BIKE LANE','PARKED IN DRIVEWAY OR ALLEY TO OBSTRUCT SIDEWALK',
                  'PARK MORE THAN 12 INCHES FROM CURB','OPP OR BESIDE CONTRUCTION SITE OBSTRUCTING TRAFFIC','STREETCAR-PARK',
                   'OBSTRUCTING ENTRANCE OF PKG GARAGE, DOOR OR GATE','PARK TO REDUCE ROADWAY TO LESS THAN 10 FEET')
residential_zone = ('RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT','PARK OVERTIME IN TIMED ZONE',
                    'RESIDENTIAL PMT PKG 3RD OFFENSE CALENDAR YEAR','RESIDENTIAL PMT PKG 2ND OFFENSE CALENDAR YEAR',
                   'PARK IN OFFICIAL PARKING PERMIT ONLY SPACE','INDIVID W/ DISABILITIES ONLY UNAUTH. USE RSRVD SP',
                   'RESERVED RESIDENTIAL SPACE PERSON W/ DISABILITIES','RESERVED RESIDENTIAL SPACE PERSONS W/ DISABILTIES')
admin_violation = ('FAILURE TO DISPLAY CURRENT TAGS','NO FRONT TAGS.','FAIL TO REPORT FOR INSPECTION.',
                  'FAIL TO DISPLAY CURRENT INSPECTION STICKERS','NO REAR TAGS.','EXPIRED INSPECTION REJECTION STICKER',
                  'IMPROPER DISPLAY OF TAGS','FAIL TO DISPLAY CURRENT INSPECTION STICKER','OBSTRUCTED TAGS.',
                  'FAIL TO PROPERLY AFFIX VALIDATION STICKERS','FAIL TO REMOVE EXPIRED RESIDENTIAL PARKING PERMIT')
rosa_violation = ('FAILURE TO SECURE DC TAGS','ROSA')

In [234]:
data['Violation Description'] = (data['Violation Description']   #Replacing values to decrease entry variety
                                      .replace(meter_violations,'Meter Violation')
                                      .replace(temp_restrictions,'Temporary Parking Restriction')
                                      .replace(illegal_location,'Illegal Parking Location')
                                      .replace(residential_zone, 'Residential Zone Violation')
                                      .replace(admin_violation, 'DMV Related Violation')
                                      .replace(rosa_violation, 'ROSA Violation')
                                      .replace(obstruct_traffic, 'Obstructing Traffic'))

In [235]:
ticket_reasons = ('Meter Violation', 'Illegal Parking Location',
       'Temporary Parking Restriction', 'Residential Zone Violation',
       'DMV Related Violation', 'ROSA Violation', 'Obstructing Traffic')

data = data.loc[data['Violation Description'].isin(ticket_reasons)]

In [236]:
#I want the top 3 states isolated and everything else summed up as 'Other'
replace_plates = data['License Plate State'].value_counts().sort_values().head(130).index
data['License Plate State'] = data['License Plate State'].replace(replace_plates,'Other')

In [237]:
data = data.dropna()

In [238]:
data['Ticket Issue Date'] = pd.to_datetime(data['Ticket Issue Date'], infer_datetime_format = True).astype(str)

#day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
#month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
#year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year
weekly = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%V-%Y')

#data['Month'] = data['Ticket Issue Date'].map(month)
#data['Day of the Week'] = data['Ticket Issue Date'].map(day_of_week)
#data['Year'] = data['Ticket Issue Date'].map(year)
data['W/Y'] = data['Ticket Issue Date'].map(weekly)

#Winter = ('12', '1', '2')
#Spring = ('3', '4', '5')
#Summer = ('6', '7','8')
#Fall = ('9', '10', '11')

#data['Season'] = (data['Month'].astype(str)
#                .replace(Winter, 'Winter')
#                .replace (Spring, 'Spring')
#                .replace (Summer, 'Summer') 
#                .replace (Fall, 'Fall'))

#data['Month'] = data['Ticket Issue Date'].map(month)

#data['Day of the Week'] = data['Day of the Week'].replace({0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})

In [239]:
data = data.dropna()

In [240]:
df = data

print(df.shape)
print()
print(df.info())
print()
print(df.isnull().sum())
print()
df.head()

(14354333, 10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14354333 entries, 0 to 14424761
Data columns (total 10 columns):
Longitude                float64
Latitude                 float64
Violation Code           object
Violation Description    object
Address                  object
License Plate State      object
Body Style               object
Block ID                 float64
Ticket Issue Date        object
W/Y                      object
dtypes: float64(3), object(7)
memory usage: 1.2+ GB
None

Longitude                0
Latitude                 0
Violation Code           0
Violation Description    0
Address                  0
License Plate State      0
Body Style               0
Block ID                 0
Ticket Issue Date        0
W/Y                      0
dtype: int64



,Longitude,Latitude,Violation Code,Violation Description,Address,License Plate State,Body Style,Block ID,Ticket Issue Date,W/Y
0,-76.990883,38.901329,P012,Illegal Parking Location,1200 BLOCK I ST NE NORTH SIDE,Other,4D,2090.0,2011-09-01 00:00:00,35-2011
1,-77.028853,38.917009,P039,Meter Violation,1300 BLOCK U ST NW NORTH SIDE,MD,4D,7819.0,2011-09-01 00:00:00,35-2011
2,-77.013880,38.888564,P036,Residential Zone Violation,200 MARYLAND AV SW,DC,TK,1186.0,2011-09-01 00:00:00,35-2011
3,-77.054309,38.926666,P172,ROSA Violation,2600 BLOCK GARFIELD ST NW NORTH SID,Other,4D,5205.0,2011-09-01 00:00:00,35-2011
4,-77.023957,38.897877,P199,Obstructing Traffic,700 9TH ST NW BLOCK EAST SIDE,DC,Other,11702.0,2011-09-01 00:00:00,35-2011


## Folium Mapping by Location

In [270]:
def ticket_count (date, violation):
    subset = df[df['W/Y'] == date]

    violations = subset.loc[df['Violation Description'] == violation]
    res_counts  = violations.groupby('Block ID').count().iloc[:,[0]]
    res_counts.columns = ['Ticket Count']
    tickets = (violations.groupby('Block ID')
               .first().join(res_counts)
               .loc[:,['Longitude','Latitude',
                       'Violation Description',
                       'Address', 'Ticket Count',
                       'W/Y', 'Ticket Issue Date']])

    
    return tickets

In [271]:
#ticket_count('07-2018', 'Meter Violation')

In [245]:
ticket_count('07-2018', 'Meter Violation').info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 575 entries, 12.0 to 100370.0
Data columns (total 7 columns):
Longitude                575 non-null float64
Latitude                 575 non-null float64
Violation Description    575 non-null object
Address                  575 non-null object
Ticket Count             575 non-null int64
W/Y                      575 non-null object
Ticket Issue Date        575 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 55.9+ KB


In [272]:
def combined(Date):
    conc = (pd.concat([ticket_count(Date ,'Illegal Parking Location'),
                      ticket_count(Date ,'Meter Violation'),
                      ticket_count(Date,"Residential Zone Violation"),
                      ticket_count(Date, 'ROSA Violation'),
                      ticket_count(Date,'Obstructing Traffic'),
                      ticket_count(Date, 'Temporary Parking Restriction'),
                      ticket_count(Date, 'DMV Related Violation')])
            .reset_index())
    return conc

In [287]:
def plot_tickets (ticket_counts):
    m = folium.Map(location=[ticket_counts['Latitude'].mean(), 
                             ticket_counts['Longitude'].mean()], 
                   zoom_start = 12,
                   tiles = ('CartoDB dark_matter'))
    
    for i in range(0,len(ticket_counts)):
        
        v = ticket_counts.iloc[i]['Violation Description']
        radius = ticket_counts.iloc[i]['Ticket Count'] ** (3/4)
        popup_text = ticket_counts.iloc[i]["Ticket Count"]
        
        if   v == 'Illegal Parking Location':
            color = '#C52121' # Red
        elif v == 'Meter Violation':
            color = '#F39C12' # Orange
        elif v == 'Residential Zone Violation':
            color = '#25C521' # Green
        elif v == 'ROSA Violation':
            color = '#D0D0D0' #  Silver
        elif v == 'Obstructing Traffic':
            color = '#BB8FCE'# Purple
        elif v == 'Temporary Parking Restriction':
            color = '#005AC6' # Blue
        else: 
            color = "#593847"  #Burgundy
            
        folium.CircleMarker(location=(ticket_counts.iloc[i]['Latitude'],
                                      ticket_counts.iloc[i]['Longitude']),
                      radius = float(radius),
                      color = color,
                      popup = str(popup_text),
                      fill = True).add_to(m)
        
    return m
        
    


In [293]:
ticket_counts = combined("30-2012")
plot_tickets(ticket_counts).save('test-30-2012.html')

In [292]:
ticket_counts.shape

(7052, 8)

In [94]:
ex = df.loc[(df['Year'] == 2018) & (df['Season'] == 'Summer')]

violations = ex['Violation Description'].unique().tolist()
blocks = []

for i in violations:
    blocks.append(ex.loc[ex['Violation Description'] == i]['Block ID'].value_counts().head().index.tolist())
    
flat_list = []

for sublist in blocks:
    for item in sublist:
        flat_list.append(item)
        
blocks = list(set(flat_list))

In [147]:
ex = ex.loc[ex['Block ID'].isin(blocks)]

In [165]:
ex['Latitude'] = round(ex['Latitude'], 5)
ex['Longitude'] = round(ex['Longitude'], 5)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [156]:
ex['Violation Description'].unique()

array(['Residential Zone Violation', 'Temporary Parking Restriction',
       'Meter Violation', 'Obstructing Traffic',
       'Illegal Parking Location', 'ROSA Violation',
       'DMV Related Violation'], dtype=object)

In [166]:
a = ex.loc[ex['Violation Description'] == 'Residential Zone Violation']
b = ex.loc[ex['Violation Description'] == 'Temporary Parking Restriction']
c = ex.loc[ex['Violation Description'] == 'Meter Violation']
d = ex.loc[ex['Violation Description'] == 'Illegal Parking Location']
f = ex.loc[ex['Violation Description'] == 'Obstructing Traffic']
g = ex.loc[ex['Violation Description'] == 'ROSA Violation']
h = ex.loc[ex['Violation Description'] == 'DMV Related Violation']

In [170]:
m = folium.Map(location=[df['Latitude'].mean() , df['Longitude'].mean()], zoom_start = 12)

"""for i in range(0,len(a)):
    folium.Circle(
        location=[a.iloc[i]['Latitude'], a.iloc[i]['Longitude']],
        popup='Residential Zone Violation',
        radius=20,
        color='#0052C2',
        fill=True,
        fill_color='#0052C2').add_to(m)

for i in range(0,len(b)):
    folium.Circle(
        location=[b.iloc[i]['Latitude'], b.iloc[i]['Longitude']],
        popup= 'Temporary Parking Restriction',
        radius=20,
        color='#08AB90',
        fill=True,
        fill_color='#08AB90').add_to(m)
    

for i in range(0,len(c)):
    folium.Circle(
        location=[c.iloc[i]['Latitude'], c.iloc[i]['Longitude']],
        popup='Meter Violation',
        radius=20,
        color='#7108AB',
        fill=True,
        fill_color='#7108AB').add_to(m)


for i in range(0,len(d)):
    folium.Circle(
        location=[d.iloc[i]['Latitude'], d.iloc[i]['Longitude']],
        popup='Illegal Parking Location',
        radius=20,
        color='#98A654',
        fill=True,
        fill_color='#98A654').add_to(m)

for i in range(0,len(f)):
    folium.Circle(
        location=[f.iloc[i]['Latitude'], f.iloc[i]['Longitude']],
        popup='Obstructing Traffic',
        radius=20,
        color='#C72B2B',
        fill=True,
        fill_color='#C72B2B').add_to(m)

for i in range(0,len(g)):
    folium.Circle(
        location=[g.iloc[i]['Latitude'], g.iloc[i]['Longitude']],
        popup= 'ROSA Violation',
        radius=20,
        color='#2BBCC7',
        fill=True,
        fill_color='#2BBCC7').add_to(m)
    

for i in range(0,len(h)):
    folium.Circle(
        location=[h.iloc[i]['Latitude'], h.iloc[i]['Longitude']],
        popup='DMV Related Violation',
        radius=20,
        color='#B91DAE',
        fill=True,
        fill_color='#B91DAE').add_to(m)
    
m.save('map.html')
"""

"for i in range(0,len(a)):\n    folium.Circle(\n        location=[a.iloc[i]['Latitude'], a.iloc[i]['Longitude']],\n        popup='Residential Zone Violation',\n        radius=20,\n        color='#0052C2',\n        fill=True,\n        fill_color='#0052C2').add_to(m)\n\nfor i in range(0,len(b)):\n    folium.Circle(\n        location=[b.iloc[i]['Latitude'], b.iloc[i]['Longitude']],\n        popup= 'Temporary Parking Restriction',\n        radius=20,\n        color='#08AB90',\n        fill=True,\n        fill_color='#08AB90').add_to(m)\n    \n\nfor i in range(0,len(c)):\n    folium.Circle(\n        location=[c.iloc[i]['Latitude'], c.iloc[i]['Longitude']],\n        popup='Meter Violation',\n        radius=20,\n        color='#7108AB',\n        fill=True,\n        fill_color='#7108AB').add_to(m)\n\n\nfor i in range(0,len(d)):\n    folium.Circle(\n        location=[d.iloc[i]['Latitude'], d.iloc[i]['Longitude']],\n        popup='Illegal Parking Location',\n        radius=20,\n        color='#9